# 学習データ・予測データの作成

## Library Import

In [1]:
# データの取り扱いに関するライブラリ
import numpy as np # 高速計算
import pandas as pd # 表データの扱い

import datetime as dt

# 可視化に関するライブラリ
import matplotlib.pyplot as plt
import japanize_matplotlib

from sklearn.model_selection import GroupKFold
import lightgbm as lgb
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

import common_func as func

In [2]:
# 自身がファイルを格納したディレクトリを指定
ROOT_DIR = '../input/'
data_definition_path = ROOT_DIR + 'data_definition.xlsx'
intermediate_path = '../output/intermediate_file/'
model_path = '../output/model/'
oof_path = '../output/oof/'
fi_path = '../output/fi/'

# スクリプトのバージョン指定
fe_ver = 3
create_tbl_ver = 2

## File Import

In [3]:
train_df = pd.read_parquet(f'{intermediate_path}train_df_fe_v{fe_ver}.parquet')
test_df = pd.read_parquet(f'{intermediate_path}test_df_fe_v{fe_ver}.parquet')

In [4]:
date_col = 'target_ym'
target_col = 'money_room'

## 空白を_で埋める

In [5]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df

In [6]:
train_df = normalize_columns(train_df)
test_df = normalize_columns(test_df)

## テーブルの分割

In [7]:
house_idx = train_df['property_group'] == 'house'
residential_idx = train_df['property_group'] == 'residential'
other_idx = train_df['property_group'] == 'other'

train_df_house = train_df[house_idx]
train_df_residential = train_df[residential_idx]
train_df_other = train_df[other_idx]

In [8]:
house_idx = test_df['property_group'] == 'house'
residential_idx = test_df['property_group'] == 'residential'
other_idx = test_df['property_group'] == 'other'

test_df_house = test_df[house_idx]
test_df_residential = test_df[residential_idx]
test_df_other = test_df[other_idx]

In [9]:
print(train_df['property_group'].value_counts(dropna=False))
print(test_df['property_group'].value_counts(dropna=False))

property_group
residential    195154
house          153456
other           15314
Name: count, dtype: Int64
property_group
residential    58834
house          48594
other           5009
Name: count, dtype: Int64


## 使用特徴量の選択

#### 共通特徴量

In [10]:
idx_key_cols = [
    'Prefecture_name',
    'City/town/village_name',
    'zone_residential_rank'
]

In [11]:
key_cols = [
    'target_ym',
    'target_year',
    'building_id',
    'unit_id',
]

In [12]:
land_base_cols = [
    # 地価
    'nearest_land_price', 'log_land_price',                                   # 最近傍地価
    'weighted_land_price_3', 'log_weighted_land_price_3',                        # 近傍3地点の距離重み付け地価
    'distance_to_landpoint_m',                          # 最近傍地点までの距離
    'land_price_yoy_nearest', 'land_price_yoy_w3',      # 前年との差分
    'land_price_dlog_nearest', 'land_price_dlog_w3',    # 前年との変化率
    
    # 土地面積
    'kukaku_area', 'kukaku_area_log',                                  # 区画面積
    'tochi_area', 'tochi_area_log',                                   # 土地面積
    'shikichi_area', 'shikichi_area_log',                                # 敷地面積

    # 理論土地価格(地価x区画面積)
    'land_theoretical_price',                           # 最近傍地価x区画面積
    # 'land_theoretical_price_weighted',                  # 近傍3地点の距離重み付け地価x区画面積
    'land_theoretical_price_within1km_mp',              # 1km以内の物件の平均価格x区画面積
    # 'proxy_land_price_calibrated',                      # station_power_sum3 x urban_score x area_per_room
    'effective_land_price',                             # land_theoretical_price_weighted or proxy_land_price
]

land_attr_cols = [
    # 用途・都市計画
    'zone_residential_rank', # 住居系の階層（低層=1, 中高層=2, 一般住居=3, 準住居=4）
    'is_urbanized_area', # 市街化区域フラグ
    'is_urban_control_area', # 市街化調整フラグ

    # 容積・建蔽
    'kenpei', # 建蔽率
    'youseki', # 容積率
    'max_floor_area', 'max_floor_area_log', # 土地面積x容積率

    # 再開発・高度利用
    'is_high_utilization_area', # 高度利用フラグ
    'is_urban_renaissance_area', # 都市再生フラグ
    'is_redevelopment_core_area', # 復興再生拠点フラグ
    'is_fireproof_area', # 防火地域
    'is_quasi_fireproof_area', # 準防火地域
    'is_land_readjustment_area', # 土地区画フラグ
    'is_special_far_area', # 特例容積率フラグ
    'is_highrise_residential_area', # 高層住居誘導地区フラグ
    'is_disaster_prevention_block', # 特定防災街区整備地区フラグ

    # 道路
    'road_len_density', 'road_len_density_gap', # 道路延長密度
    'dist_to_road_any_m', 'dist_to_road_major_m', # 道路までの距離
    'road_cnt_any_in_100m', 'road_cnt_major_in_100m', # 道路本数
    'road_cnt_any_in_300m', 'road_cnt_major_in_300m', # 道路本数
    'road_cnt_any_in_500m', 'road_cnt_major_in_500m', # 道路本数

    'is_residential_land',  # 地目が宅地フラグ
    'is_non_residential_land', # 地目が宅地ではないフラグ
    'is_flat_land',            # 平坦フラグ
    'has_slope_risk', # 傾斜値フラグ
    'is_land_area_koubo', 'is_land_area_jissoku', # 土地面積計測方式フラグ
    'has_setback',       # セットバックありフラグ
    'land_setback_log', # セットバック面積
    'setback_ratio',           # 敷地面積に対するセットバック割合

    'tag_land_premium_score',
]

land_discount_cols = [
    # 接道
    'is_no_road', # 接道なしフラグ
    'land_road_cond', # 接道状況

    # 制約・負債
    'land_constraint_score', # 土地制約・再建築リスクスコア
    'has_height_limit', # 高度制限フラグ
    'has_district_plan', # 地区計画フラグ

    # 土地余剰
    'unit_land_density', # 専有面積 / 区画面積
    'unit_land_density_over1', # 土地に対して建物が詰まりすぎ
    'density_floor_area', # 延べ床面積/土地面積
    'land_building_ratio', # 区画面積/延べ床面積(土地余り)

    'tag_land_penalty_flag',
]

land_cols = land_base_cols + land_attr_cols + land_discount_cols

In [13]:
building_base_cols = [
    # 面積
    'senyu_area', 'senyu_area_log',               # 専有面積
    'nobeyuka_area', 'nobeyuka_area_log',            # 延べ床面積
    'building_senyu_area_median',   # 建物の専有面積(log)の中央値
    'area_per_room', 'area_per_room_log',               # 1部屋あたりの専有面積
    'balcony_area', 'balcony_area_log',             # バルコニー面積
    'senyu_area_range', 'senyu_area_range_log',             # 建物の専有面積のレンジ
    'land_building_ratio',          # 区画面積/延べ床面積
    'land_building_ratio_hi',       # 区画面積/延べ床面積が20を超えたフラグ(豪邸検出フラグ)

    # 階数・構造
    'floor_count', # 建物階数(地上)
    'basement_floor_count', # 建物階数(地下)
    'room_floor', # 物件の階数
    'relative_floor', # 相対階数(地下は0)
    'is_basement', # 地下フラグ
    'structure_group', # 建物構造カテゴリ
    'building_room_floor_max', # 建物内の掲載物件の最大階数

    # 戸数
    'unit_count', 'unit_count_log', # 総戸数
    'empty_ratio', # 空室率
    'building_unit_count', # 建物内の掲載戸数
]

building_cond_cols = [
    # 築年・劣化
    'year_built', # 建築年
    'built_diff', # 築年数
    'effective_age', # リフォーム・リノベを考慮した築年数

    # リフォーム・リノベ
    'has_renovation', # 改修フラグ
    'renovation_recency', # 最後のリノベーションからの経過年
    'last_reform_year', # 最新の改修実施年
    'reform_interior_cnt',
    'reform_exterior_cnt',
    'reform_wet_cnt',
    'reform_total_cnt',
    'reform_any_flag',
    'reform_wet_heavy_flag',

    'is_area_kabeshin', 'is_area_uchinori', # 建物面積計測方式フラグ
    'window_angle', # 窓の向き
    'is_south_window', # 南向き窓フラグ
    'is_south_main_light', # 南向き主要採光面フラグ
    'south_room_ratio_proxy', # 南向き窓フラグ+南向き主要採光面フラグ

]

building_eq_cols = [
    # 設備スコア
    'premium_equipment_count', # プレミアム設備のカウント
    'semi_premium_count',
    'kitchen_upgrade_count',
    'kitchen_burner_score',
    'kitchen_grade_score',
    'kitchen_low_flag',
    'kitchen_high_flag',
    'wet_area_upgrade_count',
    'wet_area_separation_flag',
    'wet_area_grade_score',
    'toilet_upgrade_count',
    'storage_score',
    'storage_high_flag',
    'hvac_count',
    'hvac_grade_score',
    'net_ready_score',
    'security_score',
    'shared_service_score',
    'infra_modern_score',
    'has_balcony',

    # 建物性能
    'room_count', # 間取り部屋数
    # 'madori_kind_all', # 間取りの種類(TODO: 加工した方が良さそう)
    'is_one_room', 'has_k', 'has_dk', 'has_lk', 'has_ldk', 'has_storage_room', # 間取りフラグ
    'is_large_room_count', # 部屋数が多いフラグ(集合住宅は5以上、そのほかは7以上)

    # 管理
    'has_management_association', # 管理組合フラグ
    'management_total_score', # 管理スコア

    # 駐車場
    'has_parking', 'parking_nearby', 'parking_full', # 駐車場区分フラグ
    'parking_money_log', # 駐車場料金
    'parking_cost_heavy', # 2万円以上フラグ
    'parking_distance_log', # 駐車場までの距離
    'has_multiple_parking', # 駐車場複数台空きフラグ
    'parking_included_in_rent', # 駐車場代が賃料に含まれるフラグ
    'parking_fee_separate', # 駐車場代が賃料に含まれないフラグ
    

    # ペナルティ
    'infra_penalty_score',
    'hard_penalty_any',
    'discount_pressure_score',
    '専有部分設備_トイレ_トイレなし',
    '専有部分設備_浴室・洗面_バスなし',
    '専有部分設備_トイレ_共同トイレ',
    '専有部分設備_浴室・洗面_共同バス',
    '建物設備（給排水・インフラ）_汲取',
    '建物設備（給排水・インフラ）_浄化槽',
    'parking_penalty_score',
]

building_cols = building_base_cols + building_cond_cols + building_eq_cols

In [14]:
env_access_cols = [
    # 駅距離
    'door_to_station_min', 'door_to_station_min_log', # 駅までの移動時間
    'walk_distance_bin', # 駅までの移動時間のビン分割
    'access_zone',  # 生活圏カテゴリ(徒歩/バス/車)

    # 駅力
    'eki_name1_te', # 最寄り駅のターゲットエンコーディング
    'eki_name2_te', # 2つ目の最寄り駅のターゲットエンコーディング
    'rosen_name1_te', # 最寄り駅の路線のターゲットエンコーディング
    'rosen_name2_te', # 2つ目の最寄り駅の路線のターゲットエンコーディング
    'station_power_max3', # 近傍3地点の駅力スコアの最大
    'station_power_sum3', # 近傍3地点の駅力スコアの合計
]

env_liability_cols = [
    # 住みやすさスコア
    'livability_score', # 生活利便性スコア
    'score_access', # 近接道路スコア(生活利便性スコアのサブスコア)
    'score_building', # 建物スコア(生活利便性スコアのサブスコア)
    'score_room', # 部屋スコア(生活利便性スコアのサブスコア)
    'urban_score', # 都会度スコア(生活利便性スコアのサブスコア)
    'amenity_count_within_500m', 'amenity_count_within_1000m', # 周辺の施設カテゴリ数
    'env_premium_count_400m', 'env_premium_count_800m',
    'env_premium_weighted',
    
    # 医療
    'dist_hospital_m', 'dist_hospital_log', # 病院までの距離
    'hospital_1km', # 1km以内に病院があるフラグ
    'clinic_500m', # 500m以内にクリニックがあるフラグ

    # 商業
    'convenience_distance', 'convenience_distance_log', # コンビニまでの距離
    'super_distance', 'super_distance_log', # スーパーまでの距離
    'drugstore_distance', 'drugstore_distance_log', # ドラッグストアまでの距離

    # 教育
    'elem_school_500m', # 500m以内に小学校があるフラグ
    'junior_school_1km', # 1km以内に中学校があるフラグ
]

env_nearby_info_cols = [
    # 地域価格
    'Prefecture_name_te', # 都道府県ごとのターゲットエンコーディング
    'City/town/village_name_te', # 市区町村ごとのターゲットエンコーディング

    # 近傍価格
    'iqr_price_1000m', 'iqr_price_2000m', 'iqr_price_300m', 'iqr_price_500m', # 四分位範囲
    'mean_price_1000m_log', 'mean_price_1000m_mansion_log', # 平均
    'mean_price_2000m_log', 'mean_price_2000m_mansion_log', # 平均
    'mean_price_300m_log', 'mean_price_300m_mansion_log', # 平均
    'mean_price_500m_log', 'mean_price_500m_mansion_log', # 平均
    'median_price_300m_log', 'median_price_500m_log', 'median_price_1000m_log', 'median_price_2000m_log', # 中央値
    'std_price_1000m', 'std_price_2000m', 'std_price_300m', 'std_price_500m', # 標準偏差

    # 密度(どちらかというと近傍価格の信頼度を意味する)
    'count_neighbors_1000m', 'count_neighbors_1000m_house', 'count_neighbors_1000m_mansion', # 1km以内の物件数
    'count_neighbors_2000m', 'count_neighbors_2000m_house', 'count_neighbors_2000m_mansion', # カウント
    'count_neighbors_300m', 'count_neighbors_300m_house', 'count_neighbors_300m_mansion', # カウント
    'count_neighbors_500m', 'count_neighbors_500m_house', 'count_neighbors_500m_mansion', # カウント
]

env_demographic_cols = [
    'PTN_2020_nn',        # 実人口
    'pop_trend_rate_nn',  # 将来人口トレンド

    # 年齢構成（需給の質）
    'RTA_2025_nn',        # 0–14
    'RTB_2025_nn',        # 15–64
    'RTC_2025_nn',        # 65+
    'RTD_2025_nn',        # 75+
    'RTE_2025_nn',        # 80+
]

env_cols = env_access_cols + env_liability_cols + env_nearby_info_cols + env_demographic_cols

In [15]:
risk_disaster_cols = [
    # 'elev_mean', # 標高
    # 'elev_range', # 標高範囲
    # 'slope_mean', # 傾斜
    'slope_max', # 最大傾斜
    # 'slope_range', # 傾斜範囲
    'is_lowland', # 低値リスクフラグ(浸水系の代理変数)

    # 内水
    'ame_dist_log',
    'ame_depth_max_log1p',
    # 'ame_depth_effect',

    # 外水
    # 'kouzui_keikaku_in',
    'kouzui_keikaku_rank_max',
    'kouzui_keikaku_dist_m_log1p',
    # 'kouzui_saidai_in',
    'kouzui_saidai_rank_max',
    'kouzui_saidai_dist_m_log1p',
    # 'kouzui_keikaku_rank_ge3', 'kouzui_saidai_rank_ge3',

    # 土砂災害
    'keikai_dist_log1p',
    # 'keikai_within_100m',
    # 'keikai_within_300m',
    # 'keikai_within_1000m',
    # 'keikai_within_5000m',
    # 'keikai_no_near',
    'kyuusha_dist_log1p',
    # 'kyuusha_within_100m',
    # 'kyuusha_within_300m',
    # 'kyuusha_within_1000m',
    # 'kyuusha_within_5000m',
    # 'kyuusha_no_near',
    # 'dosham_dosham_A30a5_005_max_has',
    'dosham_dosham_A30a5_005_max_log1p',
    # 'dosham_dosham_A30a5_006_max_has',
    # 'dosham_dosham_A30a5_006_max_log1p',
    # 'dosham_dosham_A30a5_007_max_has',
    # 'dosham_dosham_A30a5_007_max_log1p',
    # 'dosham_in',
    # 'dosham_phen_gake',

    # 沿岸浸水
    # 'tsunami_in', 'tsunami_depth_max_m', 'tsunami_depth_rank_max',
    'tsunami_dist_log1p',
    'tsunami_depth_max_log1p',
    # 'tsunami_depth_ge1m', 'tsunami_depth_ge3m', 'tsunami_depth_ge5m', 'tsunami_depth_ge10m',
    # 'takashio_in', 'takashio_depth_max_m', 'takashio_depth_rank_max', 'takashio_dist_log1p', 
    # 'tsunami_near_500m', 'tsunami_near_1km', 'tsunami_near_3km',
    'takashio_depth_max_log1p', 
    # 'takashio_depth_ge1m', 'takashio_depth_ge3m', 'takashio_depth_ge5m', 'takashio_depth_ge10m',
    # 'takashio_near_500m', 'takashio_near_1km', 'takashio_near_3km',

    # 避難所
    'hinanjo_dist_log1p',
    # 'hinanjo_cnt_r1000', 'hinanjo_cap_sum_log1p_r1000', 'hinanjo_area_sum_log1p_r1000',
    # 'hinanjo_dist_for_suigai_flg_log1p', 'hinanjo_cnt_for_suigai_flg_r2000', 'hinanjo_cap_sum_for_suigai_flg_r2000',
    # 'hinanjo_dist_for_tsunami_flg_log1p', 'hinanjo_cnt_for_tsunami_flg_r2000', 'hinanjo_cap_sum_for_tsunami_flg_r2000',

    'discount_risk_score',
]

risk_liquidity_cols = [
    # 掲載期間
    'listing_months', 'listing_months_log', # 掲載期間

    # 物件の相対的な売れにくさ
    'empty_number', # 空き物件数
]

risk_uncertainty_cols = [
    # 持分・私道
    'mochibun_ratio', # 持分割合(欠損は100%所有とみなす)
    'mochibun_area', 'mochibun_area_log', # 持分の土地面積
    'has_mochibun', # 持分フラグ
    'land_shidou_ratio', # 私道負担割合
    'shidou_area_eff', # 私道負担面積(代表)
    'has_shidou', # 私道負担フラグ

    # 使用制限
    'genkyo_flex_score', # 現況自由度スコア
    'usable_status_score', # 引き渡し自由度スコア
    'usable_months_delay', # 引き渡しまでの期間

    # 共益費・修繕積立金
    'money_kyoueki_std', # 税込み共益費
    'kyoueki_per_m2', # 専有面積1平米あたりの共益費
    'kyoueki_per_unit', # 総戸数あたりの共益費
    'has_kyoueki', # 共益費フラグ
    'money_shuuzen', # 修繕積立金
    'shuuzen_per_m2', # 専有面積1平米あたりの修繕積立金
    'shuuzen_per_unit', # 総戸数あたりの修繕積立金
    'money_shuuzenkikin', # 修繕積立基金
    'has_shuuzen', # 修繕積立金フラグ
    'money_sonota1', # その他費用

    'cert_score',
    'cert_strong_flag',
    
    # 'liq_market_stagnation',   # 市場停滞スコア（合成）
]

risk_missing_cols = [
    'floor_count_missing',          # 建物階数
    'is_building_category_unknown', # 建物種別欠損
    'max_floor_area_missing',       # 土地面積x容積率
    'room_floor_mismatch',
    'has_senyu_area_range',
    'window_angle_mismatch',
    'investment_missing',
    'land_area_kind_missing',
    'building_area_kind_missing',
    'setback_inconsistent',
    'parking_kubun_missing',
    'parking_number_missing',
    'parking_contract_missing',
    'effective_age_missing',
]

risk_cols = risk_disaster_cols + risk_liquidity_cols + risk_uncertainty_cols + risk_missing_cols

In [16]:
market_segment_cols = [
    'is_investment', 'is_business', # 投資用物件/事業用物件フラグ
    'investment_proxy', # 投資用物件の代理変数
    'flg_new', # 新築・未入居フラグ
    'building_category', # 建物種別カテゴリ
    'luxury_mansion_flag',
    'is_rented_full_flag'
]

# 交互作用
interaction_cols = [
    # 地価 × 建物・環境
    'log_land_price_x_age_decay_30',
    'log_land_price_x_livability_score',
    'log_land_price_x_is_rcsrc',
    'log_land_price_x_is_wood',
    'log_land_price_x_effective_age',

    'log_weighted_land_price_3_x_age_decay_30',
    'log_weighted_land_price_3_x_livability_score',
    'log_weighted_land_price_3_x_is_rcsrc',
    'log_weighted_land_price_3_x_is_wood',
    'log_weighted_land_price_3_x_effective_age',

    'land_theoretical_price_x_age_decay_30',
    'land_theoretical_price_x_livability_score',
    'land_theoretical_price_x_is_rcsrc',
    'land_theoretical_price_x_is_wood',
    'land_theoretical_price_x_effective_age',
    'land_theoretical_price_x_senyu',

    'land_theoretical_price_weighted_x_age_decay_30', 
    'land_theoretical_price_weighted_x_livability_score',
    'land_theoretical_price_weighted_x_is_rcsrc', 
    'land_theoretical_price_weighted_x_is_wood',
    'land_theoretical_price_weighted_x_effective_age',  
    'land_theoretical_price_weighted_x_senyu',

    'land_theoretical_price_within1km_mp_x_age_decay_30',
    'land_theoretical_price_within1km_mp_x_livability_score',
    'land_theoretical_price_within1km_mp_x_is_rcsrc', 
    'land_theoretical_price_within1km_mp_x_is_wood',
    'land_theoretical_price_within1km_mp_x_effective_age',

    # 建物
    'senyu_area_x_built_diff',
    'area_per_room_x_built_diff',
    'senyu_to_land_value_ratio', # 専有面積 - 土地理論価格

    # 規制・道路
    'urban_control_x_land_price_log',
    'fireproof_x_max_floor_area_log',
    'district_plan_x_station_log',
    'high_util_x_youseki',
    'height_limit_x_max_floor_area_log',
    'urban_renaissance_x_station_log',
    'fireproof_x_structure',
    'road_len_density_x_no_road',
    'road_narrow_ratio_gap_x_no_road',
    'lowrise_x_station_log',
    'lowrise_x_landprice_log',

    # 地価ギャップ
    'logdiff_mean_nearest_clip', # 1km以内の物件の平均価格 - 最近傍地価
    'logdiff_median_weighted_clip', # 1km以内の物件の中央値価格 - 近傍3地点の距離重みづけ地価
    'is_land_price_gap_ratio_mean', # 1km以内の物件の平均価格が、最近傍地価の5倍以上フラグ
    'is_land_price_gap_ratio_weighted', # 1km以内の物件の中央値価格が、近傍3地点の距離重みづけ地価の5倍以上フラグ
    'land_cheap_flag', # 1km以内の物件の平均価格/最近傍地価が0.8未満
    'land_expensive_flag', # 1km以内の物件の平均価格/最近傍地価が1.2超え
    
]

other_cols = market_segment_cols + interaction_cols

#### 個別特徴量

In [17]:
residential_model_cols = [
    # floor premium
    'rel_floor_sq',
    'rel_floor_over_0_8',
    'rel_floor_over_0_9',
    'floor_premium_interaction',
    'is_top_floor',
    'is_high_floor_20',

    # maintenance burden
    'maint_per_m2_total_log',
    'maint_x_station',
    'maint_x_age',
    'maint_x_kyoueki_std_log',

    # building grade interactions
    'tower_x_relative_floor',
    'tower_x_high_floor',
    'highgrade_x_maint',
    'resort_x_elev',

    'is_luxury_fee_flag',
    'maint_x_log_land_price', #'maint_x_city_te',
    'rel_floor_x_maint',
    'mochibun_ratio_log',
    'mochibun_anomaly_flag',
    'infra_penalty_x_log_land_price',
    'hard_penalty_x_log_land_price',
    'has_shidou_x_log_land_price',

    'senyu_area_ratio_to_median',
    'area_per_room_x_senyu_diff',
    'low_price_proxy',
    'log_dist_x_livability',
]

In [18]:
house_model_cols = [
    'sin_lat', 'cos_lat', 'sin_lon', 'cos_lon',
    'shidou_area_ratio', # 土地面積に対する私道面積の比率

    'age_0_10', 'age_10_20', 'age_20_30', 'age_30_40', 'age_40_60', 'age_60_plus', # 築年数 piecewise
    'age_40_over_x_is_wood', 'age_60_over_x_is_wood', # 木造 × 築古
    'age_40_over_x_no_renov', 'age_60_over_x_no_renov', # リフォーム無し × 築古
    'no_renovation_flag',

    'theoretical_building_value', # 理論建物価値
    'land_dominance_score', 'land_dominance_score_log', # 土地支配スコア
    'land_dominant_flag', # 土地支配スコアが2以上

    'nb_build_cnt_300m', 'nb_build_cnt_500m', 'nb_build_cnt_1000m', 'nb_build_cnt_2000m', # 近傍物件数
    'nb_build_cnt_500m_log',

    'geohash6_te_logy', # geohashのTE
    'dlog_w3_x_log_land_price',
    'dlog_w3_x_land_dom',
    'dlog_w3_x_age40',

    'nobeyuka_area_x_age_40_over', 'nobeyuka_area_x_age_60_over',
    'log_land_price_x_nb_build',
    # 'land_x_road_density',
    'house_discount_score',

    'nbhd_price_level', 'nbhd_price_local_premium'
    
]

In [19]:
other_model_cols = [    
    'nb_build_cnt_500m', 'nb_build_cnt_1000m',

    'geohash6_te_logy',

    'floor_area_per_floor_log', 
    'land_tp_w_log_x_max_floor_area_log',
    'penalty_any_flag',
]

In [20]:
# risk_disaster_cols_for_residential = [
#     'riverflood_rank',
#     'inlandflood_bucket',
#     'riverflood_is_flood',
#     'inlandflood_is_flood',
# ]

#### 特徴量の定義

In [21]:
house_use_cols = idx_key_cols + key_cols + land_cols + building_cols + env_cols + risk_cols + other_cols + house_model_cols
residential_use_cols = idx_key_cols + key_cols + land_cols + building_cols + env_cols + risk_cols + other_cols + residential_model_cols
other_use_cols = idx_key_cols + key_cols + land_cols + building_cols + env_cols + risk_cols + other_cols + other_model_cols

In [22]:
def unique_preserve_order(cols: list[str]) -> list[str]:
    seen = set()
    out = []
    for c in cols:
        if c not in seen:
            seen.add(c)
            out.append(c)
    return out

In [23]:
house_use_cols = unique_preserve_order(house_use_cols)
residential_use_cols = unique_preserve_order(residential_use_cols)
other_use_cols = unique_preserve_order(other_use_cols)

## 出力

In [24]:
train_df_house[house_use_cols + [target_col]].to_parquet(f'{intermediate_path}train_df_house_v{create_tbl_ver}.parquet')
train_df_residential[residential_use_cols +  [target_col]].to_parquet(f'{intermediate_path}train_df_residential_v{create_tbl_ver}.parquet')
train_df_other[other_use_cols +  [target_col]].to_parquet(f'{intermediate_path}train_df_other_v{create_tbl_ver}.parquet')

test_df_house[house_use_cols].to_parquet(f'{intermediate_path}test_df_house_v{create_tbl_ver}.parquet')
test_df_residential[residential_use_cols].to_parquet(f'{intermediate_path}test_df_residential_v{create_tbl_ver}.parquet')
test_df_other[other_use_cols].to_parquet(f'{intermediate_path}test_df_other_v{create_tbl_ver}.parquet')

In [ ]:
# ===== LightGBM（residential） =====
# --- OOF MAPE ---
# main_city  | 0.121730
# mid_city   | 0.140349
# other      | 0.150698
# --- HO MAPE ---
# main_city  | 0.118958
# mid_city   | 0.138846
# other      | 0.129870
# OOF MAPE (all): 0.140710
# HO  MAPE (all): 0.130253

# [HPO-2STAGE] PHASE2 NEW BEST: trial=19 logMAE=0.119535 | OOF_MAPE=0.121038 | HO_MAPE=0.122709
# [HPO-2STAGE] PHASE2 NEW BEST: trial=19 logMAE=0.136767 | OOF_MAPE=0.139089 | HO_MAPE=0.142003
# [HPO-2STAGE] PHASE2 NEW BEST: trial=22 logMAE=0.147756 | OOF_MAPE=0.150670 | HO_MAPE=0.134966

In [ ]:
# ===== LightGBM（house） =====
# --- OOF MAPE ---
# low        | 0.174478
# mid        | 0.181946
# high       | 0.192228
# --- HO MAPE ---
# low        | 0.156586
# mid        | 0.157676
# high       | 0.165346
# OOF MAPE (all): 0.184585
# HO  MAPE (all): 0.161271

# [HPO-2STAGE] PHASE2 NEW BEST: trial=21 logMAE=0.168848 | OOF_MAPE=0.174515 | HO_MAPE=0.168822
# [HPO-2STAGE] PHASE1 NEW BEST: trial=18 logMAE=0.176986 | OOF_MAPE=0.182483 | HO_MAPE=0.172668
# [HPO-2STAGE] PHASE2 NEW BEST: trial=16 logMAE=0.185098 | OOF_MAPE=0.189755 | HO_MAPE=0.174317

In [ ]:
# ===== LightGBM（other） =====
# --- OOF MAPE ---
# all        | 0.245816
# --- HO MAPE ---
# all        | 0.200887
# OOF MAPE (all): 0.245816
# HO  MAPE (all): 0.200887

# 内水を追加
# [HPO-2STAGE] PHASE2 NEW BEST: trial=23 logMAE=0.231427 | OOF_MAPE=0.242584 | HO_MAPE=0.220234

# 内水と外水を追加
# [HPO-2STAGE] PHASE2 NEW BEST: trial=19 logMAE=0.230491 | OOF_MAPE=0.241132 | HO_MAPE=0.222364

# ラスト
# [HPO-2STAGE] PHASE2 NEW BEST: trial=29 logMAE=0.231342 | OOF_MAPE=0.242621 | HO_MAPE=0.222551
# [HPO-2STAGE] PHASE2 NEW BEST: trial=14 logMAE=0.230836 | OOF_MAPE=0.241870 | HO_MAPE=0.224911